In [ ]:
import pandas as pd
import numpy as np
import sqlite3
import datetime
from typing import List, Tuple, Dict, Any, Iterable
import jaconv
import csv

In [ ]:
columns_name = [
    'D番号', '氏名', 'カナ氏名', '職員番号', '生年月日', 
    '職種コード', '職制コード1', '職制1', '職制コード2', '職制2', '診療科コード'
]
df_new_comer = pd.read_csv("data/nurse_newcomer.csv", dtype=object, usecols=columns_name)

In [ ]:
dct = {
    'D番号': "D_number",
    '氏名': "kanji_name",
    'カナ氏名': "kana_name",
    '職員番号': "staff_number", 
    '漢字氏名': "kanji_name", 
    'カタカナ': "kana_name", 
    '生年月日': "birth_date",
    '職種コード': "job_code",
    '職制コード1': "job_system1_code",
    '職制1': "job_system1",
    '職制コード2': "job_system2_code",
    '職制2': "job_system2",
    '診療科コード': "dept_code",
}
df_new_comer.rename(columns=dct, inplace=True)

In [ ]:
def name_splitter(id_: Iterable, name: Iterable, new_columns: list, for_kana: bool=False) -> pd.DataFrame:
    """氏名を姓と名に分ける

    Args:
        id_ (Iterable): マージするためのID
        name (Iterable): 氏名
        new_columns (list): 新しいカラム名
        for_kana (bool, optional): 全角カナを半角カナに変換するかどうか. Defaults to False.

    Returns:
        pd.DataFrame: 姓名を分けたDataFrame
    """
    l = []
    for uid, name in zip(id_, name):
        if type(name) == str:
            name = name.replace('　　', ' ')
            name = name.replace('　 ', ' ')
            name = name.replace(' 　', ' ')
            name = name.replace('　', ' ')
            name = name.split(" ")
        else:
            name = [name] # nanの対応
        
        if for_kana:
            name = [jaconv.z2h(n) for n in name]

        length = len(name)
        if length >= 3: # 3つ以上の場合は、最初の1つを姓、残りを名に分ける
            name = [name[0], (" ").join(name[1:])]
        elif length == 1: # 1つの場合は、名を空欄にする
            name = name + [np.nan]
        l.append([uid, name[0], name[1]])

    df_new = pd.DataFrame(l, columns=new_columns, dtype=object)
    # df_new.set_index('D_number', drop=True, inplace=True)
    return df_new

In [ ]:
df_new_comer.job_system2 = df_new_comer.job_system2.map(
    lambda x: ''.join(
        chr(ord(c) - 0xFEE0)
        if ('Ａ' <= c <= 'Ｚ') or ('ａ' <= c <= 'ｚ') or ('０' <= c <= '９')
        else c
        for c in x
    )
)

In [ ]:
df_new_comer_kanji = name_splitter(df_new_comer.D_number, df_new_comer.kanji_name, ['D_number', 'family_name_kanji', 'first_name_kanji'])
df_new_comer_kana = name_splitter(df_new_comer.D_number, df_new_comer.kana_name, ['D_number', 'family_name_kana', 'first_name_kana'], for_kana=True)
df_new_comer_merge = pd.merge(df_new_comer, df_new_comer_kanji, on='D_number', how='left')
df_new_comer_merge = pd.merge(df_new_comer_merge, df_new_comer_kana, on='D_number', how='left')

In [ ]:
df_new_comer_merge.drop(columns=['kanji_name', 'kana_name'], inplace=True)

In [ ]:
df_card_data = pd.read_csv("data/nyutai_data_20250328.csv", dtype=object)
df_card_data = df_card_data[['UID', 'staff_number']]

In [ ]:
df_new_comer_merge = pd.merge(df_new_comer_merge, df_card_data, on='staff_number', how='left')

In [ ]:
def reorder_columns(df: pd.DataFrame, column_order: list) -> pd.DataFrame:
    """
    指定されたカラム順に DataFrame の列を並び替え、
    存在しないカラムは全て NaN の列として追加する。
    
    Parameters:
        df (pd.DataFrame): 元のDataFrame
        column_order (list): 並び替えたいカラム名のリスト

    Returns:
        pd.DataFrame: 並び替え後のDataFrame
    """
    new_df = pd.DataFrame()

    for col in column_order:
        if col in df.columns:
            new_df[col] = df[col]
        else:
            new_df[col] = np.nan  # 存在しないカラムはNaN列にする

    return new_df

In [ ]:
def open_dict(file_path:str, reversed:bool=False) -> Dict[str, str]:
    dct = {}
    with open(file_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            if len(row) >= 2:
                if reversed:
                    dct[row[1]] = row[0]
                else:
                    dct[row[0]] = row[1]
    return dct

In [ ]:
job_system1_code = open_dict("data/job_system1_code_list.csv", reversed=True)
job_system2_code = open_dict("data/job_system2_code_list.csv", reversed=True)

In [ ]:
df_new_comer_merge.job_system1_code = df_new_comer_merge.job_system1.map(job_system1_code)
df_new_comer_merge.job_system2_code = df_new_comer_merge.job_system2.map(job_system2_code)

In [ ]:
dct = {
    "D_number": "職員番号", # NaN
    "D_number_fired": "退職時職員番号", # NaN
    "family_name_kanji": "氏名(漢字)姓",
    "first_name_kanji": "氏名(漢字)名",
    "family_name_kana": "氏名(半角カナ)姓",
    "first_name_kana": "氏名(半角カナ)名",
    "family_name_roman": "氏名(ローマ字)姓", # NaN
    "first_name_roman": "氏名(ローマ字)名", # NaN
    "sex_code": "性別コード", # NaN
    "sex": "性別", # NaN
    "birth_date": "生年月日", # NaN
    "recruit_date": "採用日", # 2025/4/1
    "issue_date": "発令日", # NaN
    "employment_condition_code": "雇用条件コード", # NaN
    "employment_condition": "雇用条件", # NaN
    "hr_code": "人事コード(雇用条件)", # NaN
    "employment_condition_reason_code": "雇用条件選択理由(コード)", # NaN
    "employment_condition_reason": "雇用条件選択理由", # NaN
    "hr_code_employment_condition_reason": "人事コード(雇用条件選択理由)", # NaN
    "working_hours_one_day_hour": "勤務時間数(1日)_時間", # NaN
    "working_hours_one_day_minute": "勤務時間数(1日)_分", # NaN
    "working_hours_first_half_hour": "勤務時間数(前半)_時間", # NaN
    "working_hours_first_half_minute": "勤務時間数(前半)_分", # NaN
    "working_hours_second_half_hour": "勤務時間数(後半)_時間", # NaN
    "working_hours_second_half_minute": "勤務時間数(後半)_分", # NaN
    "working_hours_week_hour": "勤務時間数(週間)_時間", # NaN
    "working_hours_week_minute": "勤務時間数(週間)_分", # NaN
    "working_dates_per_week": "週所定労働日数", # NaN
    "job_system2_code": "所属コード",
    "job_system2": "所属",
    "hr_code_job_system2": "人事コード(所属)", # NaN
    "hr_code_job_system2_1": "人事コード(階層1)", # NaN
    "hr_code_job_system2_2": "人事コード(階層2)", # NaN
    "hr_code_job_system2_3": "人事コード(階層3)", # NaN
    "assign_dept_code": "配属部署コード", # NaN
    "assign_dept": "配属部署", # NaN
    "hr_code_assign_dept": "人事コード(配属部署)", # NaN
    "job_system1_code": "職種コード", # NaN
    "job_system1": "職種",
    "hr_code_job_system1": "人事コード(職種)", # NaN
    "position_code": "役職コード", # NaN
    "position": "役職", # NaN
    "hr_code_position": "人事コード(役職)", # NaN
    "job_exp_year": "他施設での経験_年", # NaN
    "job_exp_month": "他施設での経験_月", # NaN
    "recruit_date_org": "所属団体採用日", # NaN
    "staff_number": "給与番号",
    "spare1": "予備1", # NaN
    "spare2": "予備2", # NaN
    "spare3": "予備3", # NaN
    "spare4": "予備4", # NaN
    "spare5": "予備5", # NaN
    "spare6": "予備6", # NaN
    "spare7": "予備7", # NaN
    "spare8": "予備8", # NaN
    "mobile_auth_id": "モバイル認証用ID", # NaN
    "UID": "Felicaカード番号",
    "user_registration": "ユーザ登録", # 1
    "user_id_not_use": "ユーザID", # NaN
    "password": "パスワード", # NaN
}
df_new_comer_merge = reorder_columns(df_new_comer_merge, list(dct.keys()))
df_new_comer_merge.user_registration = 1
df_new_comer_merge.recruit_date = "2025/4/1"
df_new_comer_merge.position_code = "0006"
df_new_comer_merge.position = "一般"

df_new_comer_merge.rename(columns=dct, inplace=True)

In [ ]:
df_new_comer_merge.to_csv("results/nurse_newcomer_20250401.csv", index=False, header=True)
df_new_comer_merge.to_excel("results/nurse_newcomer_20250401.xlsx", index=False, header=True)